# Customer.IO Objects and Relationships Management

This notebook demonstrates how to manage objects and relationships in Customer.IO using the utils library.
Objects allow you to represent entities like companies, accounts, products, or any other data model that relates to your users.

## Setup and Imports

Import the necessary functions and initialize the Customer.IO client.

In [ ]:
# Setup and imports
import os
from datetime import datetime
from utils.api_client import CustomerIOClient
from utils.object_manager import (
    create_object,
    update_object,
    delete_object,
    create_relationship,
    delete_relationship
)
from utils.exceptions import CustomerIOError, ValidationError

# Initialize client
API_KEY = os.getenv('CUSTOMERIO_API_KEY', 'your-api-key-here')
client = CustomerIOClient(api_key=API_KEY, region='us')

print("Customer.IO object management functions loaded successfully")

## Understanding Objects in Customer.IO

Objects in Customer.IO represent entities that can have relationships with users.
Common use cases include:
- Companies/accounts in B2B SaaS
- Products in e-commerce
- Courses in education platforms
- Projects in collaboration tools

Each object has:
- **object_id**: Unique identifier for the object
- **object_type_id**: Type of object (e.g., "1" for companies, "2" for products)
- **traits**: Attributes describing the object

## Creating Objects

Create an object and establish a relationship with a user in one operation.

In [ ]:
# Example: Create a company object
company_data = {
    "name": "Acme Corporation",
    "industry": "Technology",
    "size": "50-100",
    "plan": "enterprise",
    "founded": 2015,
    "website": "https://acme.example.com"
}

try:
    result = create_object(
        client=client,
        user_id="user_001",
        object_id="company_acme",
        traits=company_data,
        object_type_id="1"  # Using "1" for companies
    )
    print("Company object created successfully")
    print(f"Result: {result}")
except CustomerIOError as e:
    print(f"Error creating company: {e}")

In [ ]:
# Example: Create a product object
product_data = {
    "name": "Premium Widget",
    "sku": "WDG-001",
    "price": 99.99,
    "category": "Hardware",
    "in_stock": True,
    "description": "High-quality widget for professionals"
}

try:
    result = create_object(
        client=client,
        user_id="user_002",
        object_id="product_wdg001",
        traits=product_data,
        object_type_id="2"  # Using "2" for products
    )
    print("Product object created successfully")
except CustomerIOError as e:
    print(f"Error creating product: {e}")

## Updating Objects

Update object attributes while maintaining relationships.

In [ ]:
# Update company information
updated_company_data = {
    "size": "100-500",  # Company grew!
    "plan": "enterprise_plus",
    "revenue_tier": "10M-50M",
    "last_contract_renewal": "2024-01-15"
}

try:
    result = update_object(
        client=client,
        user_id="user_001",
        object_id="company_acme",
        traits=updated_company_data,
        object_type_id="1"
    )
    print("Company updated successfully")
except CustomerIOError as e:
    print(f"Error updating company: {e}")

In [ ]:
# Update product with timestamp
product_update = {
    "price": 89.99,  # Price reduction
    "on_sale": True,
    "sale_ends": "2024-12-31"
}

try:
    result = update_object(
        client=client,
        user_id="user_002",
        object_id="product_wdg001",
        traits=product_update,
        object_type_id="2",
        timestamp=datetime.now()
    )
    print("Product updated with sale information")
except CustomerIOError as e:
    print(f"Error updating product: {e}")

## Managing Relationships

Create and delete relationships between users and objects without modifying object attributes.

In [ ]:
# Create relationship between another user and existing company
try:
    result = create_relationship(
        client=client,
        user_id="user_003",  # New user
        object_id="company_acme",  # Existing company
        object_type_id="1"
    )
    print("User user_003 added to company_acme")
except CustomerIOError as e:
    print(f"Error creating relationship: {e}")

In [ ]:
# Create multiple relationships for a user
# Example: User belongs to multiple projects
projects = [
    {"id": "project_alpha", "name": "Project Alpha"},
    {"id": "project_beta", "name": "Project Beta"},
    {"id": "project_gamma", "name": "Project Gamma"}
]

for project in projects:
    try:
        # First create the project object
        create_object(
            client=client,
            user_id="user_004",
            object_id=project["id"],
            traits={"name": project["name"], "status": "active"},
            object_type_id="3"  # Using "3" for projects
        )
        print(f"Created project: {project['name']}")
    except CustomerIOError as e:
        print(f"Error with project {project['name']}: {e}")

## Deleting Relationships

Remove relationships between users and objects without deleting the object itself.

In [ ]:
# Remove user from company
try:
    result = delete_relationship(
        client=client,
        user_id="user_001",
        object_id="company_acme",
        object_type_id="1"
    )
    print("User removed from company")
except CustomerIOError as e:
    print(f"Error deleting relationship: {e}")

## Deleting Objects

Completely remove an object from Customer.IO.

In [ ]:
# Delete a project object
try:
    result = delete_object(
        client=client,
        object_id="project_gamma",
        object_type_id="3"
    )
    print("Project deleted successfully")
except CustomerIOError as e:
    print(f"Error deleting object: {e}")

## Real-World Example: B2B SaaS Account Management

Complete workflow for managing company accounts and user relationships in a B2B SaaS application.

In [ ]:
# Scenario: Onboarding a new B2B customer
def onboard_b2b_customer(client, company_info, admin_user, team_members):
    """Onboard a new B2B customer with company and team setup."""
    
    # Step 1: Create company object with admin user
    try:
        create_object(
            client=client,
            user_id=admin_user["id"],
            object_id=company_info["id"],
            traits=company_info["traits"],
            object_type_id="1"
        )
        print(f"Created company: {company_info['traits']['name']}")
        
        # Step 2: Add team members to company
        for member in team_members:
            create_relationship(
                client=client,
                user_id=member["id"],
                object_id=company_info["id"],
                object_type_id="1"
            )
            print(f"Added {member['name']} to company")
            
        return True
        
    except CustomerIOError as e:
        print(f"Onboarding failed: {e}")
        return False

In [ ]:
# Example usage of onboarding function
new_company = {
    "id": "company_techstartup",
    "traits": {
        "name": "Tech Startup Inc",
        "plan": "growth",
        "industry": "SaaS",
        "size": "10-50",
        "trial_ends": "2024-02-15"
    }
}

admin = {"id": "user_admin_001", "name": "Jane Smith"}
team = [
    {"id": "user_dev_001", "name": "John Developer"},
    {"id": "user_sales_001", "name": "Sarah Sales"}
]

onboard_b2b_customer(client, new_company, admin, team)

## Real-World Example: E-commerce Product Relationships

Managing user relationships with products for personalization and recommendations.

In [ ]:
# Track user product interactions
def track_product_interaction(client, user_id, product_id, interaction_type):
    """Track various user interactions with products."""
    
    product_traits = {
        "last_interaction": datetime.now().isoformat(),
        "interaction_type": interaction_type,
        "interaction_count": 1  # In real app, increment existing count
    }
    
    try:
        # Create/update product relationship
        update_object(
            client=client,
            user_id=user_id,
            object_id=product_id,
            traits=product_traits,
            object_type_id="2"  # Products
        )
        print(f"Tracked {interaction_type} for {product_id}")
        return True
    except CustomerIOError as e:
        print(f"Tracking failed: {e}")
        return False

In [ ]:
# Example: Track various product interactions
user_id = "user_shopper_001"
interactions = [
    ("product_laptop_001", "viewed"),
    ("product_laptop_001", "added_to_cart"),
    ("product_mouse_001", "viewed"),
    ("product_keyboard_001", "purchased")
]

for product_id, action in interactions:
    track_product_interaction(client, user_id, product_id, action)

## Error Handling and Validation

Proper error handling for object operations.

In [ ]:
# Example: Validation errors
invalid_operations = [
    # Empty object ID
    lambda: create_object(client, "user123", "", {"name": "Test"}, "1"),
    # Invalid traits
    lambda: create_object(client, "user123", "obj123", "not a dict", "1"),
    # Empty traits
    lambda: create_object(client, "user123", "obj123", {}, "1"),
    # Invalid object type ID
    lambda: create_object(client, "user123", "obj123", {"name": "Test"}, "zero")
]

for i, operation in enumerate(invalid_operations):
    try:
        operation()
    except ValidationError as e:
        print(f"Validation error {i+1}: {e}")

## Best Practices

### Object Type Organization
- Use consistent object_type_id values across your application
- Document your object type mappings (e.g., 1=companies, 2=products)
- Consider using constants or enums in production code

### Object ID Naming
- Use meaningful, unique identifiers
- Consider prefixing with type (e.g., "company_", "product_")
- Match your internal database IDs when possible

### Trait Management
- Keep traits flat and simple when possible
- Use consistent naming conventions
- Include timestamps for time-sensitive data

### Performance
- Use batch operations for bulk object creation
- Consider rate limits when processing many objects
- Cache object data locally when appropriate

## Next Steps

Now that you understand object and relationship management, explore:

- **04_device_management.ipynb** - Managing user devices for push notifications
- **05_batch_operations.ipynb** - Bulk operations for objects and relationships
- **02_event_tracking.ipynb** - Track events related to object interactions